# KEN4157 - Reinforcement Learning - Cliff Walking Gymnasium
If you opened this notebook in Google Colab, we recommend to start by saving a copy of the notebook in your own Google Drive, such that you can save any of your changes and experiments.

## Installing & Importing Modules
We will start by installing importing some modules that will likely be useful for your assignment(s). This includes [Gymnasium](https://gymnasium.farama.org/), which is a framework containing many popular RL environments (a successor to the original Gym API from OpenAI).

In [1]:
!pip install gymnasium

import gymnasium as gym
import itertools
import math
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.tri as tri
import matplotlib.colors as color


import numpy as np
import seaborn as sns

from tqdm import tqdm

# Make plots look nice
sns.set()
sns.set_context("notebook")
sns.set_style("whitegrid")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.4 MB/s eta 0:00:00


## Setting up the Cliff Walking Environment
Here, we'll set up the Cliff Walking environment, and have a look at how to interact with it according to the Gym API.

**Optional**:
- For a description and documentation of the environment, see: https://gymnasium.farama.org/environments/toy_text/cliff_walking/
- For the implementation of the environment, see: https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/toy_text/cliffwalking.py

In [2]:
env = gym.make('CliffWalking-v0')

action_space = env.action_space
obs_space = env.observation_space
print(f"Action space: {action_space}")
print(f"Observation space: {obs_space}")

# We'll define a few strings here, because proper names are sometimes nicer
# to work with (print) than arbitrary indices
ACTION_NAMES = ["Move Up", "Move Right", "Move Down", "Move Left"]

print(f"There are {action_space.n} different actions in this environment.")
for i in range(action_space.n):
  print(f"Action {i} = {ACTION_NAMES[i]}")
print(f"There are {obs_space.n} discrete states in this environment.")

Action space: Discrete(4)
Observation space: Discrete(48)
There are 4 different actions in this environment.
Action 0 = Move Up
Action 1 = Move Right
Action 2 = Move Down
Action 3 = Move Left
There are 48 discrete states in this environment.


## Visualisation Code
Some functions here that let us nicely visualise Q-value tables for the Cliff Walking environment. No need to touch any of this, but you should run the code block, such that you can use the functions later on.

In [3]:
# Thanks to: https://stackoverflow.com/a/66050636/6735980
def triangulation_for_triheatmap(num_cols, num_rows):
    xv, yv = np.meshgrid(np.arange(-0.5, num_cols), np.arange(-0.5, num_rows))  # vertices of the little squares
    xc, yc = np.meshgrid(np.arange(0, num_cols), np.arange(0, num_rows))  # centers of the little squares
    x = np.concatenate([xv.ravel(), xc.ravel()])
    y = np.concatenate([yv.ravel(), yc.ravel()])
    cstart = (num_cols + 1) * (num_rows + 1)  # indices of the centers

    trianglesN = [(i + j * (num_cols + 1), i + 1 + j * (num_cols + 1), cstart + i + j * num_cols)
                  for j in range(num_rows) for i in range(num_cols)]
    trianglesE = [(i + 1 + j * (num_cols + 1), i + 1 + (j + 1) * (num_cols + 1), cstart + i + j * num_cols)
                  for j in range(num_rows) for i in range(num_cols)]
    trianglesS = [(i + 1 + (j + 1) * (num_cols + 1), i + (j + 1) * (num_cols + 1), cstart + i + j * num_cols)
                  for j in range(num_rows) for i in range(num_cols)]
    trianglesW = [(i + (j + 1) * (num_cols + 1), i + j * (num_cols + 1), cstart + i + j * num_cols)
                  for j in range(num_rows) for i in range(num_cols)]
    return [tri.Triangulation(x, y, triangles) for triangles in [trianglesN, trianglesE, trianglesS, trianglesW]]

# Thanks to: https://stackoverflow.com/a/66050636/6735980
def create_qvals_heatmap(q_vals_up, q_vals_right, q_vals_down, q_vals_left):
    NUM_COLS = 12
    NUM_ROWS = 4

    values = [q_vals_up , q_vals_right, q_vals_down, q_vals_left]
    triangul = triangulation_for_triheatmap(NUM_COLS, NUM_ROWS)
    fig, ax = plt.subplots()

    imgs = [ax.tripcolor(t, val.ravel(), cmap='RdYlGn',
                        norm=color.SymLogNorm(linthresh=20.0, linscale=1.0,
                                               vmin=np.min(values),
                                               vmax=np.max(values), base=10)
                        )
            for t, val in zip(triangul, values)]
    cbar = fig.colorbar(imgs[0] , ax=ax)

    ax.set_xticks(range(NUM_COLS))
    ax.set_yticks(range(NUM_ROWS))
    ax.invert_yaxis()
    ax.margins(x=0, y=0)
    ax.set_aspect('equal', 'box')  # square cells
    plt.tight_layout()
    plt.show()

def plot_returns_per_episode(returns_per_episode):
    fig, ax = plt.subplots()
    plt.plot([episode for episode in range(len(returns_per_episode))], returns_per_episode, label="Returns per Episode")
    ax.set_xlabel("Episode")
    ax.set_ylabel(f"Returns")
    ax.legend()
    plt.show()


## Random Value Function & Policy
In the below code block, we'll:
- fill up a Q(s, a) table with completely random numbers
- run through a bunch of steps with a random policy
- demonstrate how the above functions can be used to visualise this random Q-table and policy

Afterwards, you can implement your own (non-random) algorithms and use the plotting functions in the same way.

In [4]:
random_q_table = np.random.random((obs_space.n, action_space.n))
returns_per_episode = []

observation, info = env.reset()
current_episode_returns = 0.0

for step in tqdm(range(100_000)):
    action = action_space.sample()

    new_obs, reward, terminated, truncated, info = env.step(action)
    current_episode_returns = current_episode_returns + reward

    if terminated or truncated:
        # episode ended: store data and reset to new episode
        returns_per_episode.append(current_episode_returns)
        current_episode_returns = 0.0
        observation, info = env.reset()
    else:
        observation = new_obs

# Plot returns we got for each completed episode
plot_returns_per_episode(returns_per_episode)

# Plot heatmap from our Q-table (which in this case is just random numbers)
q_vals_up = random_q_table[:, 0]
q_vals_right = random_q_table[:, 1]
q_vals_down = random_q_table[:, 2]
q_vals_left = random_q_table[:, 3]
create_qvals_heatmap(q_vals_up, q_vals_right, q_vals_down, q_vals_left)

100%|██████████| 100000/100000 [00:03<00:00, 31402.64it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-3-1c7614a9f92d>:34: UserWarning: All values for SymLogScale are below linthresh, making it effectively linear. You likely should lower the value of linthresh. 
  cbar = fig.colorbar(imgs[0] , ax=ax)


## Your Code
Your own code should probably go below here!

# Q learning

In [5]:
# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
num_episodes = 2000  # Number of episodes for training

# Function to choose an action always exploiting the learned values
def choose_action(state):
    return np.argmax(random_q_table[state])  # Exploit learned values

# Q-learning algorithm
random_q_table = np.random.random((obs_space.n, action_space.n))
returns_per_episode = []  # To store total rewards per episode

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_rewards = 0

    while not done:
        action = choose_action(state)
        next_state, reward, done, truncated, info = env.step(action)

        # Q-learning update
        old_value = random_q_table[state, action]
        next_max = np.max(random_q_table[next_state])

        new_value = old_value + alpha * (reward + gamma * next_max - old_value)
        random_q_table[state, action] = new_value

        total_rewards += reward
        state = next_state

    returns_per_episode.append(total_rewards)


plot_returns_per_episode(returns_per_episode)

# Plot heatmap from our Q-table
q_vals_up = random_q_table[:, 0]
q_vals_right = random_q_table[:, 1]
q_vals_down = random_q_table[:, 2]
q_vals_left = random_q_table[:, 3]
create_qvals_heatmap(q_vals_up, q_vals_right, q_vals_down, q_vals_left)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-3-1c7614a9f92d>:34: UserWarning: All values for SymLogScale are below linthresh, making it effectively linear. You likely should lower the value of linthresh. 
  cbar = fig.colorbar(imgs[0] , ax=ax)


# SARSA with epsilon-greedy policy

In [6]:
# Parameters for SARSA
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 2000  # Number of episodes for training

# Function to choose an action using epsilon-greedy policy
def choose_action(state):
    if np.random.rand() < epsilon:
        return env.action_space.sample()  # Explore action space
    else:
        return np.argmax(random_q_table[state])  # Exploit learned values

# SARSA algorithm
random_q_table = np.random.random((obs_space.n, action_space.n))
returns_per_episode = []  # To store total rewards per episode

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_rewards = 0
    action = choose_action(state)
    while not done:

        next_state, reward, done, truncated, info = env.step(action)
        next_action = choose_action(next_state)
        # SARSA update

        old_value = random_q_table[state, action]

        new_value = old_value + alpha * (reward + gamma * random_q_table[next_state, next_action] - old_value)
        random_q_table[state, action] = new_value

        total_rewards += reward
        state = next_state
        action = next_action

    returns_per_episode.append(total_rewards)


plot_returns_per_episode(returns_per_episode)

# Plot heatmap from our Q-table
q_vals_up = random_q_table[:, 0]
q_vals_right = random_q_table[:, 1]
q_vals_down = random_q_table[:, 2]
q_vals_left = random_q_table[:, 3]
create_qvals_heatmap(q_vals_up, q_vals_right, q_vals_down, q_vals_left)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SARSA with boltzmann policy

In [7]:
# Parameters for SARSA
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
num_episodes = 2000  # Number of episodes for training
temperature = 1  # Temperature for boltzmann policy

# Function to choose an action using boltzmann policy
def choose_action(state):
    q_values = random_q_table[state]
    probabilities = np.exp(q_values / temperature) / np.sum(np.exp(q_values / temperature))
    return np.random.choice(np.arange(len(q_values)), p=probabilities)


# SARSA algorithm
random_q_table = np.random.random((obs_space.n, action_space.n))
returns_per_episode = []  # To store total rewards per episode

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_rewards = 0
    action = choose_action(state)
    while not done:

        next_state, reward, done, truncated, info = env.step(action)
        next_action = choose_action(next_state)
        # SARSA update

        old_value = random_q_table[state, action]

        new_value = old_value + alpha * (reward + gamma * random_q_table[next_state, next_action] - old_value)
        random_q_table[state, action] = new_value

        total_rewards += reward
        state = next_state
        action = next_action

    returns_per_episode.append(total_rewards)


plot_returns_per_episode(returns_per_episode)

# Plot heatmap from our Q-table
q_vals_up = random_q_table[:, 0]
q_vals_right = random_q_table[:, 1]
q_vals_down = random_q_table[:, 2]
q_vals_left = random_q_table[:, 3]
create_qvals_heatmap(q_vals_up, q_vals_right, q_vals_down, q_vals_left)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# True online TD-lambda

In [ ]:
from os import wait
import numpy as np
# Parameters for TD(lambda)
alpha = 0.01  # Learning rate
gamma = 0.95  # Discount factor
num_episodes = 500  # Number of episodes for training
epsilon_start  = 1.0  # Epsilon for epsilon-greedy policy
epsilon_min = 0.01   # Minimum epsilon valu
lambda_val = 0.95  # Lambda value for TD(lambda)
epsilon_decay = (epsilon_start - epsilon_min) / num_episodes

num_actions = env.action_space.n
num_states = env.observation_space.n

# Epsilon-greedy policy for action selection
def epsilon_greedy_policy(state, w, num_actions, env, epsilon):
    """
    Select an action using an epsilon-greedy policy based on current state.

    :param state: The current state.
    :param w: The weight vector for the value function.
    :param num_actions: The number of possible actions.
    :param env: The environment, used to check action space if necessary.
    :param epsilon: The probability of choosing a random action.
    :return: The selected action.
    """
    q_values = np.zeros(num_actions)
    for action in range(num_actions):
        feature_vector = get_feature_vector(state, action, False)  # Assume non-terminal for action-value computation
        q_values[action] = np.dot(w.T, feature_vector)

    if np.random.rand() < epsilon:
        # Exploration: Randomly select an action
        action = np.random.choice(num_actions)
    else:
        # Exploitation: Select the action with the maximum Q-value
        action = np.argmax(q_values)

    return action


def get_feature_vector(state, action, is_terminal):
    """ Returns a one-hot encoded feature vector for the state-action pair, zero if terminal. """
    feature_vector = np.zeros(num_states * num_actions)
    if is_terminal:
        return feature_vector  # Return zero vector if terminal
    index = state * num_actions + action
    feature_vector[index] = 1
    return feature_vector


# Initialize value-function weights, eligibility trace
w = np.zeros(num_states * num_actions)  # Linear weight vector
z = np.zeros(num_states * num_actions)  # Eligibility trace vector
returns_per_episode = []  # To store total rewards per episode

# True Online TD(λ) for Sarsa(λ)
for episode in range(num_episodes):
    done = False
    state, _ = env.reset()
    x = get_feature_vector(state, 0, done)
    V_old = 0
    total_rewards = 0
    epsilon = max(epsilon_min, epsilon_start - episode * epsilon_decay)  # Update epsilon with decay


    while not done:
        action = epsilon_greedy_policy(state, w, num_actions, env, epsilon)
        next_state, reward, done, truncated, info = env.step(action)
        next_action = epsilon_greedy_policy(next_state, w, num_actions, env, epsilon)

        x = get_feature_vector(state, action, done)
        x_prime = get_feature_vector(next_state, next_action, done)

        # Compute values
        V = np.dot(w.T, x)
        V_prime = np.dot(w.T, x_prime)
        delta = reward + gamma * V_prime - V

        # Update traces and weights
        z = gamma * lambda_val * z + (1 - alpha * gamma * lambda_val * np.dot(z.T, x)) * x
        w = w + alpha * (delta + V - V_old) * z - alpha * (V - V_old) * x
        V_old = V_prime
        x = x_prime

        # Update state
        state = next_state
        total_rewards += reward


    if episode % (num_episodes/10) == 0:
        print(f"Episode {episode}: Total rewards = {total_rewards}")
        # Plot heatmap from our Q-table
        Q = w.reshape((num_states, num_actions))
        q_vals_up = Q[:, 0]
        q_vals_right = Q[:, 1]
        q_vals_down = Q[:, 2]
        q_vals_left = Q[:, 3]
        create_qvals_heatmap(q_vals_up, q_vals_right, q_vals_down, q_vals_left)
    returns_per_episode.append(total_rewards)


plot_returns_per_episode(returns_per_episode)

# Plot heatmap from our Q-table
Q = w.reshape((num_states, num_actions))
q_vals_up = Q[:, 0]
q_vals_right = Q[:, 1]
q_vals_down = Q[:, 2]
q_vals_left = Q[:, 3]
create_qvals_heatmap(q_vals_up, q_vals_right, q_vals_down, q_vals_left)

Episode 0: Total rewards = -69299


<IPython.core.display.Javascript object>

Episode 50: Total rewards = -3979


<IPython.core.display.Javascript object>

Episode 100: Total rewards = -3262


...

Episode 300: Total rewards = -571


<ipython-input-3-1c7614a9f92d>:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


<IPython.core.display.Javascript object>

Episode 350: Total rewards = -57


<IPython.core.display.Javascript object>